# Fusion des données démographiques FairFace

Ce notebook ajoute les colonnes `age`, `gender` et `race` au fichier `meta.csv` pour permettre l'affichage des statistiques démographiques dans l'application.

In [1]:
import pandas as pd
from pathlib import Path

## 1. Chargement des données

In [2]:
# Chemins
base_path = Path("../data")
meta_path = base_path / "processed" / "meta.csv"
fairface_path = base_path / "raw" / "fairface" / "fairface.csv"

# Chargement
meta = pd.read_csv(meta_path)
fairface = pd.read_csv(fairface_path)

print(f"Meta: {len(meta)} lignes")
print(f"FairFace: {len(fairface)} lignes")

Meta: 85022 lignes
FairFace: 97698 lignes


In [3]:
# Aperçu
display(meta.head())
display(fairface.head())

,index,filename,path,source,identity
0,0,1.jpg,..\data\processed\fairface_faces\1.jpg,fairface,NaN
1,1,10.jpg,..\data\processed\fairface_faces\10.jpg,fairface,NaN
2,2,100.jpg,..\data\processed\fairface_faces\100.jpg,fairface,NaN
3,3,1000.jpg,..\data\processed\fairface_faces\1000.jpg,fairface,NaN
4,4,10000.jpg,..\data\processed\fairface_faces\10000.jpg,fairface,NaN


,file,age,gender,race,service_test,split,img_path
0,train/1.jpg,50-59,Male,East Asian,True,train,..\data\raw\fairface\train\1.jpg
1,train/2.jpg,30-39,Female,Indian,False,train,..\data\raw\fairface\train\2.jpg
2,train/3.jpg,3-9,Female,Black,False,train,..\data\raw\fairface\train\3.jpg
3,train/4.jpg,20-29,Female,Indian,True,train,..\data\raw\fairface\train\4.jpg
4,train/5.jpg,20-29,Female,Indian,True,train,..\data\raw\fairface\train\5.jpg


## 2. Préparation pour la jointure

Le fichier `meta.csv` utilise `filename` (ex: `1.jpg`) alors que `fairface.csv` utilise `file` (ex: `train/1.jpg`). On extrait le nom du fichier pour la correspondance.

In [4]:
# Extraire le nom du fichier depuis fairface
fairface["filename"] = fairface["file"].apply(lambda x: x.split("/")[-1])
fairface[["file", "filename"]].head()

,file,filename
0,train/1.jpg,1.jpg
1,train/2.jpg,2.jpg
2,train/3.jpg,3.jpg
3,train/4.jpg,4.jpg
4,train/5.jpg,5.jpg


## 3. Fusion

In [5]:
# Sélectionner uniquement les colonnes nécessaires
fairface_subset = fairface[["filename", "age", "gender", "race"]].drop_duplicates(subset="filename")

# Fusion avec meta
meta_enriched = meta.merge(
    fairface_subset,
    on="filename",
    how="left"
)

print(f"Lignes après fusion: {len(meta_enriched)}")
print(f"Colonnes: {meta_enriched.columns.tolist()}")

Lignes après fusion: 85022
Colonnes: ['index', 'filename', 'path', 'source', 'identity', 'age', 'gender', 'race']


In [6]:
# Vérifier la fusion
print("Statistiques de fusion:")
print(f"  - Total: {len(meta_enriched)}")
print(f"  - Avec démographie (FairFace): {meta_enriched['age'].notna().sum()}")
print(f"  - Sans démographie (our_faces): {meta_enriched['age'].isna().sum()}")

display(meta_enriched.head(10))

Statistiques de fusion:
  - Total: 85022
  - Avec démographie (FairFace): 84987
  - Sans démographie (our_faces): 35


,index,filename,path,source,identity,age,gender,race
0,0,1.jpg,..\data\processed\fairface_faces\1.jpg,fairface,NaN,50-59,Male,East Asian
1,1,10.jpg,..\data\processed\fairface_faces\10.jpg,fairface,NaN,30-39,Male,Middle Eastern
2,2,100.jpg,..\data\processed\fairface_faces\100.jpg,fairface,NaN,20-29,Female,East Asian
3,3,1000.jpg,..\data\processed\fairface_faces\1000.jpg,fairface,NaN,30-39,Male,White
4,4,10000.jpg,..\data\processed\fairface_faces\10000.jpg,fairface,NaN,50-59,Male,White
5,5,10001.jpg,..\data\processed\fairface_faces\10001.jpg,fairface,NaN,more than 70,Female,Latino_Hispanic
6,6,10002.jpg,..\data\processed\fairface_faces\10002.jpg,fairface,NaN,30-39,Male,White
7,7,10003.jpg,..\data\processed\fairface_faces\10003.jpg,fairface,NaN,3-9,Male,East Asian
8,8,10004.jpg,..\data\processed\fairface_faces\10004.jpg,fairface,NaN,40-49,Male,Indian
9,9,10005.jpg,..\data\processed\fairface_faces\10005.jpg,fairface,NaN,20-29,Male,Southeast Asian


## 4. Sauvegarde

In [7]:
# Sauvegarder le fichier enrichi
meta_enriched.to_csv(meta_path, index=False)
print(f" Fichier sauvegardé: {meta_path}")

 Fichier sauvegardé: ..\data\processed\meta.csv


## 5. Vérification

In [8]:
# Recharger pour vérifier
meta_check = pd.read_csv(meta_path)
print("Colonnes du fichier final:")
print(meta_check.columns.tolist())

print("\n Distribution par genre (FairFace uniquement):")
print(meta_check[meta_check['source'] == 'fairface']['gender'].value_counts())

print("\n Distribution par âge (FairFace uniquement):")
print(meta_check[meta_check['source'] == 'fairface']['age'].value_counts())

Colonnes du fichier final:
['index', 'filename', 'path', 'source', 'identity', 'age', 'gender', 'race']

 Distribution par genre (FairFace uniquement):
gender
Male      44966
Female    40021
Name: count, dtype: int64

 Distribution par âge (FairFace uniquement):
age
20-29           25072
30-39           18803
40-49           10537
3-9             10237
10-19            8937
50-59            6105
60-69            2708
0-2              1768
more than 70      820
Name: count, dtype: int64


C:\Users\ulric\AppData\Local\Temp\ipykernel_8164\3434696770.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_check = pd.read_csv(meta_path)
